## Shear Moment Diagrams

The relationship amongst load $w$, shear $V$, and moment $M$ in a beam can be described using integrals:

$$\Delta V = \int w(x) dx $$

$$\Delta M = \int V(x) dx $$

This gives us an opportunity to do some simple discretizations of these equations and quickly create load, shear, and moment diagrams.<br><br>

Let's start with a linear load functions, where the load is either constant across the beam, or increases at a constant rate.  We'll also limit ourselves to cantilevered beams, because we know that the shear and moment at the end of the beam is zero, and so can avoid the complications of including the supporting forces.  <br><br>

<img src = https://github.com/AugustanaPEA/ENGR_311/raw/main/Images/Cantilever_beam_right.PNG width = 400>

We'll define our parameters first: the length of the beam, the load at $x=0$ (the base of the beam), and the slope of the changing load.  Then we'll add a function to define the load at a given position $x$:



In [ ]:
import numpy as np
import pandas as pd


# Length of the beam in meters
L = 1.0

# Linear load (positive load is in downward direction)
init = 0.0      # in N
slope = 1.0      # in N/m

# Linear load function
def linear_load(slope,init,x):
    return slope*x + init

Now we begin to discretize our equations.  What does this mean?  We'll divide our beam into many small (but not infinitesimal) parts, and we'll add up the load and shear in these small parts in order to calculate the integrals above.  In doing so, we'll quantify the shear and moment at every point on the beam. <br><br>

Notice that in the cell below our array of position values `x_array` starts at the end of the beam $L$ and goes to 0: this is because we know the values of the shear and moment at the end of the beam, and will integrate (i.e. add up) from there.  The next cell outputs `x_array` so you can see what that array looks like.

In [ ]:
# Decide on the number of steps and calculate the width of each step
# More steps means more accuracy but more computational expense
# 'delta' is the width of each step
steps = 1001
delta = L/(steps-1)

# Create an array of our x values from L to 0
# Then create a DataFrame (i.e. a table) to hold our values for w, V, and M
x_array = np.linspace(L,0,steps)
results = pd.DataFrame(index=x_array, columns=['Load','Shear','Moment'], dtype=object)
print(x_array)

Now we need to start to populate the DataFrame with the appropriate values.  Start with the first position at $x = 0$:

In [ ]:
# Populate the load values for every position x
# and set the shear and moment values at x = L
results.Load = linear_load(slope,init,x_array)
V = 0
M = 0

# Enter the first line into our DataFrame with the
# initial values of w, V, and M
# Notice that the first row of results.Load is at
# the end of the beam (that is, at x = L)
results.iloc[0] = [results.Load.iloc[0],V,M]
print(results.Load)

Now we'll loop through every $x$ value and add the change in shear and moment to our DataFrame. <br><br>

Notice that in lines 3 and 5 we are finding the average value of the load and shear (respectively) on the two sides of the "step", and multiplying that average by the width of the step:

$$V_{after~ this~ step} = V_{before~this~step} +\frac{w_i + w_{i+1}}{2}\Delta x$$ 

This is called a *discretization scheme*, and is an example of how we can calculate a continuous integral using a computer.  <br>

<img src = https://github.com/AugustanaPEA/ENGR_311/raw/main/Images/Cantilever_beam_disc_right.PNG width = 300>

<br>

It's important that we calculate shear first so that we'll have a shear value at $x_{i+1}$ when we go to calculate the moment:

In [ ]:
# Loop through each "step" and add to the value of V and M
# This is how we are integrating using discrete values
for i in range(len(x_array)-1):
    # Add the new weight load on this step to the shear total
    V = V + (results.Load.iloc[i]+results.Load.iloc[i+1])/2.0 * delta
    results.iloc[i+1] = [results.Load.iloc[i+1],V,0]
    # Add the new shear on this step to the moment total
    M = M - (results.Shear.iloc[i]+results.Shear.iloc[i+1])/2.0 * delta
    results.iloc[i+1] = [results.Load.iloc[i+1],V,M]

Now we can plot the three columns of our DataFrame:

In [ ]:
results.plot(subplots=True,layout=[1,3],figsize=[15,3]);

Test the code by putting in the conditions of the cantilevered beam in lecture "1.2 Load Distribution Diagrams Example" and checking that the results match the answer there.

### Exercise 1

It's a little more complicated to deal with point loads.  But if we check at every step in the loop to find if there is a point load, we can do it. <br><br>

We'll start by adding to our parameters: we'll store any point results in a Series called `points`.  Our point loads here are a 10.0 N load at both the midpoint ($x=0.5$ m) and the end ($x=0.99$ m) of the beam:

In [ ]:
L = 1.0          # in m
init = 10.0      # in N
slope = 5.0      # in N/m
points = pd.Series(index=[0.5,0.99], data=[10.0,10.0])
steps = 101
delta = L/(steps-1)
x_array = np.linspace(L,0,steps)

Now we need a way to add the effect of these point loads.  The function below takes the load data created by `linear_load()`, and adds to it any point loads.  Notice that the point loads get divided by `delta` in the function: this is because they will later be multiplied by `delta` when we calculate shear:

In [ ]:
def point_loads(x_array,points,delta,results):
    # Loop through every "step" in the x-direction
    for i in range(len(x_array)-1):
        # Check to see if there is a point load in this step
        for p in points.index:
            if p <= x_array[i] and p > x_array[i+1]:
                # Add the point load to the load function 
                # 'Load_image' is added here as fourth element in the DataFrame
                # to make our plots more readable later on...
                results.iloc[i] = [results.Load.iloc[i] + points[p]/delta,0,0, 
                                 results.Load_image.iloc[i] + points[p]]
    return results

Now we can create our load function using both `linear_load()` and `point_loads()`.  Once we've done that, we can calculate shear and moment just as we did above (`Load_image` is added just to make our plots more readable when we're done):

In [ ]:
# Set up the DataFrame with initial values
results = pd.DataFrame(index=x_array, columns=['Load','Shear','Moment','Load_image'], dtype=object)
results.Load = linear_load(slope,init,x_array)
results.Load_image = linear_load(slope,init,x_array)
results.Shear = 0.0
results.Moment = 0.0
results = point_loads(x_array,points, delta,results)
V = 0
M = 0
results.iloc[0] = [results.Load.iloc[0],V,M,results.Load_image.iloc[0]]

# Loop through the steps and integrate the load and shear
for i in range(len(x_array)-1):
    V = V + (results.Load.iloc[i]+results.Load.iloc[i+1])/2.0 * delta
    results.iloc[i+1] = [results.Load.iloc[i+1],V,0,results.Load_image.iloc[i+1]]
    M = M - (results.Shear.iloc[i]+results.Shear.iloc[i+1])/2.0 * delta
    results.iloc[i+1] = [results.Load.iloc[i+1],V,M,results.Load_image.iloc[i+1]]

And now we'll plot our results.  We're using `pyplot` here rather than our usual `plot` because it is more versatile: it allows us to plot just 3 of the 4 parts of the DataFrame on the same plot:

In [ ]:
# Plot the load, shear, and moment diagrams
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 3, figsize=(15, 3))
axs[0].plot(results.Load_image)
axs[1].plot(results.Shear, color='red')
axs[2].plot(results.Moment.index, results.Moment, color='green')
fig.suptitle('Load, Shear, and Moment Diagrams');
axs[0].set_xlabel('Length (m)')
axs[1].set_xlabel('Length (m)')
axs[2].set_xlabel('Length (m)')
axs[0].set_xlabel('Load (N/m or N)')
axs[1].set_xlabel('Shear')
axs[2].set_xlabel('Moment')

✅ ✅  Explain in your own words why the shear diagram jumps suddenly at $x=0.5$ m.  Why does the point load increase the shear so drastically?

✅ ✅ Answer here.

### Exercise 2

Add the necessary parameters to the code in exercise 2 to confirm your answer to HW question 1.8 (and leave these plots when you turn the HW in).  

This code is set up to have the support on the right side.  So solve the problem with that in mind.  Notice that the sign of the shear plot will be different than your HW because of the sign conventions (problem 1.8 has the support on the right side of the beam, rather than the left).<br><br>

<img src = https://github.com/AugustanaPEA/ENGR_311/raw/main/Images/cantilever_HW_problem.PNG width = 300>